In [1]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
from pydantic import BaseModel, ValidationError
from typing import List
from google.genai.types import GenerateContentConfig, Tool
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from IPython.display import display, HTML, Markdown
import io
import json
import re
import typing_extensions as typing

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
api_key = os.getenv('GEMINI_API_KEY')

# Initialize the Google AI client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model='gemini-2.0-flash-exp', contents='What is your name?'
)
print(response.text)




I am a large language model, trained by Google.



In [2]:
MODEL = "gemini-2.0-flash-exp"

In [3]:

COMPANY = 'Proximus'

In [4]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

In [5]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates and metrics to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

Add to the report a link to the most recent company logo, the company colors in rgb, a summary of companies who offer services to them, a comprehensive summary of the key employees and their roles in the company, a summary of departments of the company. 

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_k=1, top_p=1, safety_settings=safety_settings)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)


Okay, I will create a company report for Proximus. Here's my plan:

1.  **Company Overview:** I'll start by gathering general information about Proximus, such as its industry, headquarters location, and a brief history.
2.  **Logo and Colors:** I'll find the most recent company logo and identify its official colors, expressing them in RGB format.
3.  **Service Providers:** I'll research companies that provide services to Proximus.
4.  **Key Employees:** I'll identify key employees and their roles within the company.
5.  **Company Departments:** I'll outline the main departments within Proximus.
6.  **Recent News and Updates:** I'll look for any recent news or updates about the company.

Now, let's start with the research.
Okay, I have gathered the necessary information. Here's the company report for Proximus:

---
**Company Report: Proximus**

**1. Company Overview:**

Proximus Group is a leading provider of digital services and communication solutions, operating primarily in Belgium a

**Proximus Key Employees and Roles**

In [6]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up the company website, company news, updates, metrics, linkedin pages, common news articles, financial reports and public records to write a research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

The report should only contain a comprehensive summary of the employees in the company. 

The report should contain, the complete first name and family name, the role that they have within the company and the department that they make part of.

Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found.

Give me as much grounded names that you can find with your research and put them all in the report.

The report should be written in this style example:

Use this JSON schema:

Proximus_employees = {'first_name':str, 'family_name':str, 'role':str, 'department':str}
Return: list[Proximus_employees]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)



```json
[
    {
        "first_name": "Guillaume",
        "family_name": "Boutin",
        "role": "Chief Executive Officer",
        "department": "Leadership Squad"
    },
    {
        "first_name": "Stefaan",
        "family_name": "De Clerck",
        "role": "Chairman of the Board",
        "department": "Board of Directors"
    },
     {
        "first_name": "Jim",
        "family_name": "Casteele",
        "role": "Chief Consumer Market Officer",
        "department": "Leadership Squad"
    },
    {
        "first_name": "Antonietta",
        "family_name": "Mastroianni",
        "role": "Chief Digital & IT Officer",
         "department":"Leadership Squad"
    },
     {
        "first_name": "Mark",
        "family_name": "Reid",
        "role": "Finance Lead",
        "department": "Leadership Squad"
    },
    {
        "first_name": "Geert",
        "family_name": "Standaert",
        "role": "Network & Wholesale Lead",
        "department":"Leadership Squad"
    },
     

In [7]:
import re
import json

def save_json_from_string(input_string, output_filename):
    """
    Extracts JSON content from a string and saves it to a JSON file with the specified name.

    :param input_string: The input string containing JSON data within brackets.
    :param output_filename: The name of the output JSON file (e.g., "data.json").
    :return: True if the operation was successful, False otherwise.
    """
    # Step 1: Extract the JSON content within the brackets
    result = re.search(r'\[.*\]', input_string, re.DOTALL)

    if result:
        extracted_content = result.group(0)  # Get the matched content

        try:
            # Step 2: Parse the extracted content into a Python object (list/dict)
            json_data = json.loads(extracted_content)

            # Step 3: Write the JSON data to a file
            with open(output_filename, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, indent=4)  # Save with pretty formatting

            print(f"JSON data has been successfully written to '{output_filename}'.")
            return True  # Operation was successful
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except Exception as e:
            print(f"An error occurred while writing the file: {e}")
    else:
        print("No content found within brackets.")

    return False  # Operation failed


In [8]:
save_json_from_string(response.text, 'proximus_employees.json')

JSON data has been successfully written to 'proximus_employees.json'.


True

**Departments Proximus**

In [9]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up the company website, company news, updates, metrics, linkedin pages, common news articles, financial reports and public records to write a research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Proximus_departments = {'department':str, 'subdivision':list[str]}
Return: list[Proximus_departments]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)

print(type(response.text))


```json
[
    {
        "department": "CEO's Department",
        "subdivision": []
    },
    {
        "department": "Consumer Market",
        "subdivision": []
    },
     {
        "department": "Enterprise Market",
        "subdivision": ["Proximus NXT"]
    },
    {
        "department": "Corporate Affairs",
         "subdivision": []
    },
    {
        "department":"Network & Wholesale",
        "subdivision": []
    },
    {
        "department": "Customer Operations",
        "subdivision": []
    },
    {
        "department": "Human Capital",
        "subdivision": []
    },
    {
        "department":"Digital & IT",
        "subdivision": []
    },
    {
        "department":"Finance",
        "subdivision": []
    },
     {
        "department": "Proximus Global",
        "subdivision": ["BICS", "Telesign", "Route Mobile"]
    },
    {
        "department": "Proximus Media House",
         "subdivision": ["Pickx", "Pickx+", "Pickx+ Sports", "Pickx Live"]
    },
    {
  

In [10]:
save_json_from_string(response.text, 'proximus_departments.json')

JSON data has been successfully written to 'proximus_departments.json'.


True

**Company Colors**

In [11]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up the company website, company news, updates, metrics, linkedin pages, common news articles, financial reports and public records to write a research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

Do not include speculative colors and focus only on the main ones.

The report should be written in this style example:

Use this JSON schema:

company_colors = {'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]},{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}
Return: list[company_colors]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)

```json
[
    {
        "color_name_1": "Primary Purple",
        "hex_code_1": "#5C2D91",
        "rgb_code_1": [92, 45, 145]
    },
    {
        "color_name_2": "White",
        "hex_code_2": "#FFFFFF",
        "rgb_code_2": [255, 255, 255]
    },
    {
        "color_name_3": "Black",
        "hex_code_3": "#000000",
         "rgb_code_3": [0, 0, 0]
    }
]
```


In [12]:
save_json_from_string(response.text, 'proximus_colors.json')

JSON data has been successfully written to 'proximus_colors.json'.


True

**Service providers**

In [13]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up the company website, company news, updates, metrics, linkedin pages, common news articles, financial reports and public records to write a research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

List the main service providers of the company. Include any known suppliers, contractors, technology service providers, or any other third-party companies that the company relies on. 
Provide the names of the service providers, type and description of the services they provide to the company.

The report should be written in this style example:

Use this JSON schema:

company_service = {"provider":str,"service":str,"type":str]}
Return: list[company_service]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5,safety_settings=safety_settings)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)

```json
[
    {
        "provider": "Scarlet",
        "service": "Telecommunication services",
        "type": "Subsidiary using Proximus infrastructure"
    },
    {
        "provider": "Mobile Vikings",
        "service": "Telecommunication services",
        "type": "Brand of Proximus"
    },
     {
        "provider": "Tango",
        "service": "Telecommunication services",
        "type": "Brand of Proximus in Luxembourg"
    },
    {
        "provider": "Telindus Luxembourg",
         "service": "Telecommunication and ICT services",
        "type": "Brand of Proximus in Luxembourg"
    },
    {
        "provider": "Telindus Netherlands",
        "service": "Telecommunication and ICT services",
        "type": "Brand of Proximus in the Netherlands"
    },
    {
        "provider": "BICS",
        "service": "International communication enabler",
        "type": "Affiliate of Proximus, provides global voice and mobile data services"
    },
    {
        "provider": "Telesign",
  

In [14]:
save_json_from_string(response.text, 'proximus_providers.json')

JSON data has been successfully written to 'proximus_providers.json'.


True